In [1]:
from time import time
import random

In [2]:
epochs_duration = []
epochs_duration.append(time())
epochs_duration.append(time())

### Evaluation function 

In [3]:
def evaluate_solution(solution, books_scores):
    books_scanned = set()
    points = 0
    number_of_libraries = solution[0]
    for i in range(1, number_of_libraries+1):
        number_of_books = solution[i][1]
        for j in range(number_of_books):
            book_to_add = solution[i][2][j]
            if book_to_add not in books_scanned:
                books_scanned.add(book_to_add)
    solution_score = 0
    for i in books_scanned:
        solution_score += books_scores[i]
        
    return solution_score

### Get complete solution from list of solutions

In [4]:
def get_complete_solution(solution_indices, libraries, days_number, books_scores):
    time_pointer = days_number
    used_books = set()
    complete_solution = [len(solution_indices)]
    for i in solution_indices:
        time_pointer -= libraries[i][0][1]
        books_per_day = libraries[i][0][2]
        available_books = list(libraries[i][1])
        particular_solution = [i, 0, []]
        available_books.sort(key = (lambda x: books_scores[x]))
        for x in available_books:
            if len(particular_solution[2]) > time_pointer * books_per_day:
                break
            if x not in used_books:
                particular_solution[2].append(x)
                particular_solution[1] += 1
                used_books.add(x)
        particular_solution[2] = tuple(particular_solution[2])
        complete_solution.append(tuple(particular_solution))
    return tuple(complete_solution)

### Initial population

In [5]:
def get_random_solution(libraries_number, days_number, libraries, books_scores):
    libraries_to_use = list(libraries)
    solution_indices = []
    time_pointer = 0
    while True:
        random_index = random.choice(list(range(len(libraries_to_use))))
        random_library = libraries_to_use[random_index]
        if time_pointer + random_library[0][1] > days_number:
            break                                 
        solution_indices.append(random_index)
        time_pointer += random_library[0][1]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)
        
    
def get_initial_population(population_size, libraries_number, days_number, libraries, books, timestamp):
    population = []
    for i in range(int(population_size / 2)):
        population.append(get_random_solution(libraries_number, days_number, libraries, books))
        population.append(get_random_solution(libraries_number, days_number, libraries, books))
        if time() - timestamp >= 150:
            return (i * 2, population)
    return (population_size, population)

### Mutation

In [6]:
def mutate(solution, libraries, days_number, books_scores):
    solution_indices = [solution[i][0] for i in range(1, len(solution))]
    for _ in range(3):
        nucleotide_a = random.randint(0, len(solution_indices) - 1)
        nucleotide_b = random.randint(0, len(solution_indices) - 1)
        solution_indices[nucleotide_a], solution_indices[nucleotide_b] = solution_indices[nucleotide_b], solution_indices[nucleotide_a]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)

def do_mutations(population, libraries, days_number, books_scores):
    mutated_solutions = []
    for solution in population:
        mutated_solutions.append(mutate(solution, libraries, days_number, books_scores))
    return mutated_solutions

## Crossover

In [7]:
def get_children(parent_a, parent_b, libraries, days_number, books_scores):
    parent_a_indices = [parent_a[i][0] for i in range(1, len(parent_a))]
    parent_b_indices = [parent_b[i][0] for i in range(1, len(parent_b))]
    split_point = random.randint(0, min(len(parent_a_indices), len(parent_b_indices)) - 1)
    parent_a_indices[:split_point], parent_b_indices[split_point:] = parent_b_indices[:split_point], parent_a_indices[split_point:]
    for parent in (parent_a_indices, parent_b_indices):
        time_pointer = 0
        for i in range(len(parent)):
            if time_pointer >= days_number:
                parent_a_indices = parent[:i]
                break
            time_pointer += libraries[parent[i]][1]
        time_pointer = 0
    child_a = get_complete_solution(parent_a_indices, libraries, days_number, books_scores)
    child_b = get_complete_solution(parent_a_indices, libraries, days_number, books_scores)
    return (child_a, child_b)
    

In [8]:
def get_offspring(population, libraries, days_number, books_scores):
    offspring = []
    for i in range(len(population), 2):
        offspring.append(get_children(population[i], population[i + 1], libraries, days_number, books_scores))
    return tuple(offspring)

# Main

<font color="blue">__Specify how long program is able to run here (in seconds)__</blue>

In [9]:
max_duration = 300

<font color="blue">__Specify population size here__</font>

In [10]:
population_size = 60

### Read data

<font color=blue>__Specify path to input file here__</font>

In [11]:
file = open("instances/b_read_on.txt", "r")

Load data from source file to the program

In [12]:
books_number, libraries_number, days_number = [int(x) for x in file.readline().split()]
books_scores = tuple([int(x) for x in file.readline().split()])
libraries = [None] * libraries_number
for i in range(libraries_number):
    libraries[i] = (tuple([int(x) for x in file.readline().split()]), tuple([int(x) for x in file.readline().split()]))
libraries = tuple(libraries) 

Get initial population

In [13]:
(population_size, population) = get_initial_population(population_size, libraries_number, days_number, libraries, books_scores, epochs_duration[0])

main loop - epochs <br>


In [14]:
epoch = 0
while epochs_duration[0] + max_duration >= time() + (epochs_duration[-1] - epochs_duration[-2]) * 2:
    mutations = do_mutations(population, libraries, days_number, books_scores)
    offspring = get_offspring(population, days_number, libraries, books_scores)
    population.extend(offspring)
    population.extend(mutations)
    population.sort(key = lambda x: evaluate_solution(x, books_scores), reverse = True)
    population = population[:population_size]
    epochs_duration.append(time())
    print("epoch: " + str(epoch) + "; timestamp: " + str(round(time() - epochs_duration[0], 2)) +  "; epoch's duration: ", str(round(epochs_duration[-1] - epochs_duration[-2], 2)) + "; best solution: " + str(evaluate_solution(population[0], books_scores)))
    epoch += 1

epoch: 0; timestamp: 3.28; epoch's duration:  3.28; best solution: 4262800
epoch: 1; timestamp: 5.22; epoch's duration:  1.95; best solution: 4262800
epoch: 2; timestamp: 7.24; epoch's duration:  2.02; best solution: 4268500
epoch: 3; timestamp: 9.33; epoch's duration:  2.09; best solution: 4301000
epoch: 4; timestamp: 11.48; epoch's duration:  2.15; best solution: 4321300
epoch: 5; timestamp: 13.85; epoch's duration:  2.38; best solution: 4321300
epoch: 6; timestamp: 16.11; epoch's duration:  2.25; best solution: 4359400
epoch: 7; timestamp: 18.33; epoch's duration:  2.22; best solution: 4374000
epoch: 8; timestamp: 20.54; epoch's duration:  2.21; best solution: 4374000
epoch: 9; timestamp: 22.82; epoch's duration:  2.28; best solution: 4405100
epoch: 10; timestamp: 25.01; epoch's duration:  2.19; best solution: 4420400
epoch: 11; timestamp: 27.25; epoch's duration:  2.24; best solution: 4490200
epoch: 12; timestamp: 29.65; epoch's duration:  2.4; best solution: 4490200
epoch: 13; tim

In [15]:
print(population[0][0])
for i in range(1, len(population[0])):
    print(population[0][i][0], population[0][i][1], end = " ")
    print(" ".join([str(x) for x in population[0][i][2]]))

90
82 997 82000 82001 82002 82003 82004 82005 82006 82007 82008 82009 82010 82011 82012 82013 82014 82015 82016 82017 82018 82019 82020 82021 82022 82023 82024 82025 82026 82027 82028 82029 82030 82031 82032 82033 82034 82035 82036 82037 82038 82039 82040 82041 82042 82043 82044 82045 82046 82047 82048 82049 82050 82051 82052 82053 82054 82055 82056 82057 82058 82059 82060 82061 82062 82063 82064 82065 82066 82067 82068 82069 82070 82071 82072 82073 82074 82075 82076 82077 82078 82079 82080 82081 82082 82083 82084 82085 82086 82087 82088 82089 82090 82091 82092 82093 82094 82095 82096 82097 82098 82099 82100 82101 82102 82103 82104 82105 82106 82107 82108 82109 82110 82111 82112 82113 82114 82115 82116 82117 82118 82119 82120 82121 82122 82123 82124 82125 82126 82127 82128 82129 82130 82131 82132 82133 82134 82135 82136 82137 82138 82139 82140 82141 82142 82143 82144 82145 82146 82147 82148 82149 82150 82151 82152 82153 82154 82155 82156 82157 82158 82159 82160 82161 82162 82163 82164 

In [16]:
print(evaluate_solution(population[0], books_scores))

4691900


In [17]:
print(time() - epochs_duration[0])

57.122843742370605
